<a href="https://colab.research.google.com/github/innamast/simulative_python_2/blob/main/%D0%A7%D0%B5%D1%82%D0%B2%D0%B5%D1%80%D1%82%D1%8B%D0%B9_%D0%BA%D0%B5%D0%B9%D1%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1 кейс**

**Работа с логами**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить необходимый для работы файл.

In [1]:
!wget https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log

--2025-07-27 19:18:28--  https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log
Resolving gist.github.com (gist.github.com)... 140.82.116.3
Connecting to gist.github.com (gist.github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log [following]
--2025-07-27 19:18:28--  https://gist.githubusercontent.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 459418 (449K) [text/plain]
Saving to: ‘auto_purchase.log’

auto_purchase.log   100%[===================>] 448.65K  --.-KB/s    in 0.03s   

2025-07-27 19:18:29 (13.0

Чтобы посмотреть как он выглядит выполните следующую ячейку.

In [2]:
with open('auto_purchase.log', 'r') as f:
    lines = f.readlines()

for line in lines[400:550]:
    print(line)

INFO  | 2022-11-26 00:01:03,024  | file: demon_auto_purchase_bundle.py |  line: 54 | [demon] Cегодня 2022-11-26, количество людей с автопродлением подписки: 0

INFO  | 2022-11-26 00:01:03,027  | file: demon_auto_purchase_bundle.py |  line: 57 | [demon] Выход из программы

INFO  | 2022-11-27 00:01:02,455  | file: demon_auto_purchase_bundle.py |  line: 44 | [demon] Поверяем истекающие премиум-пакеты

INFO  | 2022-11-27 00:01:02,480  | file: demon_auto_purchase_bundle.py |  line: 54 | [demon] Cегодня 2022-11-27, количество людей с автопродлением подписки: 0

INFO  | 2022-11-27 00:01:02,483  | file: demon_auto_purchase_bundle.py |  line: 57 | [demon] Выход из программы

INFO  | 2022-11-28 00:01:02,406  | file: demon_auto_purchase_bundle.py |  line: 44 | [demon] Поверяем истекающие премиум-пакеты

INFO  | 2022-11-28 00:01:02,424  | file: demon_auto_purchase_bundle.py |  line: 54 | [demon] Cегодня 2022-11-28, количество людей с автопродлением подписки: 0

INFO  | 2022-11-28 00:01:02,427  | f

### **Решения**

#### **Задача 1**

Ваша задача написать функцию `count_success_and_failure`, которая принимает на вход путь к файлу с логами и подсчитывает количество успешных продлений и ошибок при списании. Функция должна вернуть кортеж из двух значений: количества успешных попыток и неуспешных.

**Решение**

Напишите свое решение ниже

In [3]:

def count_success_and_failure(file_path):
    with open(file_path, 'r') as f:
        data = f.readlines()

    errors = sum(['ERROR'in row for row in data])
    attempts = sum(['Обновляем' in row for row in data])
    return (attempts - errors, errors)

count_success_and_failure('auto_purchase.log')

(1034, 186)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [4]:
res = count_success_and_failure('auto_purchase.log')

try:
    assert res == (1034, 186)
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 2**

Ваша задача написать функцию `auto_renewal_sub`, которая принимает на вход путь к файлу с логами и обрабатывает количество клиентов с автопродлением подписки. Мы хотим посмотреть на изменение этого показателя в динамике: посчитайте сглаженные значения с помощью метода скользящего среднего и метода медианного сглаживания.  

**Примечание:** При сглаживании берем все предыдущие значения, включая текущее, будущие значения не берем. Если в один день наблюдаем несколько записей об автопродлении - берем максимальное из имеющихся число клиентов с подпиской.

Функция должна записать в файл `auto_renewal_sub.txt` два списка, предварив их соответствущими обозначениями:

`Среднее: [2.0, 1.0, 0.67...]`

`Медиана: [2, 2, 0...]`

**Решение**

Напишите свое решение ниже

In [8]:

from datetime import date as d
from statistics import mean,median_low

def auto_renewal_sub(log_file_path):
    with open('auto_purchase.log', 'r') as f:
        logs = [line for line in f if 'Cегодня' in line]

    dates = [d.fromisoformat(row.strip().split(sep= 'Cегодня ')[1][:10]) for row in logs]
    qtys = [int(row.rsplit(sep=': ', maxsplit=1)[1].strip()) for row in logs]

    data = list(set([(date, qty) for date, qty in zip(dates, qtys)]))
    data.sort()
    # для каждой из дат
    uniq_dates = [row[0] for row in data]
    # считаем сглаженное среднее
    means = []
    for date in uniq_dates:
        qtys = [row[1] for row in data if row[0] <= date]
        avg = round(float(mean(qtys)), 2)
        means.append(avg)

    # и сглаженную медиану
    medians = []
    for date in uniq_dates:
        qtys = [row[1] for row in data if row[0] <= date]
        med = median_low(qtys)
        medians.append(med)

    # записываем в файл
    with open('auto_renewal_sub.txt', 'w') as f:
        f.write(f'Среднее: {means}\n')
        f.write(f'Медиана: {medians}')

auto_renewal_sub('auto_purchase.log')

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [9]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt

import pandas as pd

user_answer = pd.read_csv('auto_renewal_sub.txt')
correct_answer = pd.read_csv('cor_auto_renewal.txt')

--2025-07-27 20:12:09--  https://gist.github.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt
Resolving gist.github.com (gist.github.com)... 140.82.116.4
Connecting to gist.github.com (gist.github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt [following]
--2025-07-27 20:12:09--  https://gist.githubusercontent.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2431 (2.4K) [text/plain]
Saving to: ‘cor_auto_renewal.txt.1’

cor_auto_renewal.tx 100%[===================>]   2.37K  --.-KB/s    in 0s      

2025-07-27 20

In [10]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!


#### **Задача 3**

Напишите функцию `sub_renewal_by_day`, которая принимает на вход путь к файлу с логами и анализирует взаимосвязь дня продления подписки и количества продлений в этот день. Функция должна записать в файл `weekdays.txt` аналитическую записку в формате:

**`Количество обновлений подписки по дням недели:`**

**`Понедельник: 6`**

**`Вторник: 7`**

**`Среда: 8`**

**`...`**

**Решение**

Напишите свое решение ниже

In [17]:
from datetime import datetime

def sub_renewal_by_day(file_path):
  with open(file_path, 'r') as f:

    days = ["Понедельник", "Вторник", "Среда", "Четверг", \
            "Пятница", "Суббота", "Воскресенье"]
    data = {k: 0 for k in range(7)}

    lines = f.readlines()
    for line in lines:
      event_date = datetime.strptime(line[8:18], '%Y-%m-%d').weekday()
      condition = line.find('Обновляем подписку пользователю id') == -1
      if line.startswith('ERROR') and line.endswith('ошибка при списании:\n'):
        data[event_date] -= 1
      if line.startswith('INFO') and not condition:
        data[event_date] += 1

  with open('weekdays.txt', 'w') as f:
    f.write('Количество обновлений подписки по дням недели:\n')
    s = []
    for key, value in data.items():
      s.append(f'{days[key]}: {value}')
    f.write('\n'.join(s))

sub_renewal_by_day('auto_purchase.log')

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [18]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt

import pandas as pd

user_answer = pd.read_csv('weekdays.txt')
correct_answer = pd.read_csv('cor_weekdays.txt')

--2025-07-27 20:22:25--  https://gist.github.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt
Resolving gist.github.com (gist.github.com)... 140.82.116.3
Connecting to gist.github.com (gist.github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt [following]
--2025-07-27 20:22:25--  https://gist.githubusercontent.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238 [text/plain]
Saving to: ‘cor_weekdays.txt.2’

cor_weekdays.txt.2  100%[===================>]     238  --.-KB/s    in 0s      

2025-07-27 20:22:25 (11.3 MB/s) - ‘co

In [19]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!
